In [1]:
#Reading all the files (stop words, negative and positive words into 3 different dictionaries)

In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import seaborn as sns
import pandas as pd
import numpy as np
%pylab inline

import string
import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  

import bs4 as bs
import csv
import requests
import urllib
from urllib.request import urlopen

Populating the interactive namespace from numpy and matplotlib


C:\Users\nevil\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [39]:
text=''
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\Apple_India.txt","r") as apple_text:
    for line in apple_text:                         
        line = line.strip().lower()
        line = line.translate(string.punctuation)
        text += line
print(text)


bloomberg: apple loses key execs in india strugglesjul. 16, 2018 8:05 am et|about: apple inc. (aapl)|by: brandy betz, sa news editorapple (nasdaq:aapl) has lost three execs in india in recent weeks, according to bloomberg sources.the departures include the national sales and distribution chief, the commercial channels and mid-market business head, and the leader of telecom carrier sales.apple has struggled to gain traction in india due to high import tariffs and a consumer base that prefers cheaper smartphones. apple has a single-digit market share in the country.sources say apple has been slow to cultivate local business relationships and has had difficulties understanding the market.michel coulomb took over as head of indian operations last december. coulomb has been with apple since 2003 and was previously the south asia managing director.apple shares are up 0.2% premarket to $191.65.previously: apple roundup: production starts on iphone 6s in india, barclays forecasts june q (june 

In [40]:
fb_text1=''
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\facebook_slowingrevenue.txt","r") as apple_text:
    for line in apple_text:                         
        line = line.strip().lower().translate(string.punctuation)
        fb_text1 += line
print(fb_text1)

facebook -20% on guidance for slowing revenue growthjul. 25, 2018 5:40 pm et|about: facebook (fb)|by: jason aycock, sa news editorfacebook's (nasdaq:fb) slide has turned into a dive after hours, with shares now down 20% on millions of shares traded, after the conference call reveals guidance for dropping revenue growth rates.the company says it sees growth rate dropping by high single digits for the next couple of quarters.total expense growth will exceed revenue growth in 2019, the company says.share fell in the immediate aftermath of the company's q2 report, where profits beat expectations but revenues missed and some observers had hoped for higher monthly active users (actual report: 2.23b).


In [41]:
fb_text2=''
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\fb_china.txt","r") as apple_text:
    for line in apple_text:                         
        line = line.strip().lower().translate(string.punctuation)
        fb_text2 += line
print(fb_text2)

nyt: china withdraws approval for facebook unitjul. 25, 2018 10:17 am et|about: facebook (fb)|by: jason aycock, sa news editorchina has quickly pulled approval for facebook (nasdaq:fb) to open a local subsidiary, the new york times reports.a database that showed the approval has seen the registration disappear, and references to the subsidiary have been partially censored in chinese media -- and now that approval has been withdrawn, according to the report.the removal comes amid a disagreement between provincial officials in zhejiang (where facebook's local unit was to be set) and the cyberspace administration of china.previously: facebook sets up chinese subsidiary (jul. 24 2018)visit seeking alpha's premium etf screener, the best etf research tool for your portfolio


In [42]:
nr_text=''
with open("C:\\Users\\nevil\\Desktop\\new_republic_puritan_inc.txt","r") as apple_text:
    for line in apple_text:                         
        line = line.strip().lower().translate(string.punctuation)
        nr_text += line
print(nr_text)

in our days we have a new scene of superstitious impostors and heretics,” declared the scholastic robert burton, about the fierce english calvinists skeptical of the anglican compromise and known by a series of sobriquets in the unruly englands of elizabeth and james. they referred to themselves as “the godly,” but were called by others less charitable “the precise;” another epithet was “brownist,” after robert “troublechurch” browne, an early charismatic leader. they were known also as separatists, for their sporadic opposition to the anglican church, became known as puritans, for their theological severity, and later still as pilgrims, in recognition of the atlantic journey that their theology seemed to require of them, and that brought to the fertile soil of mercantilist america the ascetic seedlings of our vast middle-classness. “of those men i may conclude generally,” burton wrote in 1621, a year after the sailing of the mayflower, “that howsoever they may seem to be discreet, and

# Sentiment Analysis

In [68]:
##Reading stop words into stop_words dictionary
stop_words={}
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\stop_words.txt",'r') as sw:    ##opening stop_words.txt file
    for line in sw:                        
        line =line.strip().lower()               ##stripping any blank spaces from the beginning & end of line
        stop_words[line] = 0             ## Entering the word to stop_words dictionary

In [71]:
##Reading negative words into negative_words dictionary
negative_words={}
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\positive-words.txt",'r') as nw:          ##opening negative_words.txt file
    for line in nw:
        line = line.strip().lower()                         ##stripping any blank spaces from the beginning & end of line
        negative_words[line] = 0                     ## Entering the word to negative_words dictionary

In [72]:
##Reading positive words into positive_words dictionary
positive_words={}
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\negative-words.txt",'r') as pw:   ##opening postive_words.txt file
    for line in pw:
        line = line.strip().lower()                  ##stripping any blank spaces from the beginning & end of line
        positive_words[line] = 0             ## Entering the word to postive_words dictionary

In [46]:
##Reading uncertain words into uncertain_words dictionary
uncertain_words={}
with open("C:\\Users\\nevil\\Documents\\GitHub\\Practicum\\LM_uncertain_list.txt",'r') as pw:   ##opening postive_words.txt file
    for line in pw:
        line = line.strip().lower()                  ##stripping any blank spaces from the beginning & end of line
        uncertain_words[line] = 0             ## Entering the word to postive_words dictionary

In [73]:
# Check if words have been loaded or not
print(len(stop_words))                  
print(len(negative_words))
print(len(positive_words))
print(len(uncertain_words))

119
2007
4783
297


In [74]:
# Read the  file and identify number of positive and negative words while taking care of stop words
positive_count = 0
negative_count = 0
uncertain_count = 0
##defining clean(word) function for the purpose of removing special characters
def clean(word):
    symbols = ['!','@','#','$','%','_','^','&','*','(',')','-','+','=','?']
    for symbol in symbols:
        word = word.replace(symbol,'')
    return word

path = "C:\\Users\\nevil\\Desktop\\new_republic_puritan_inc.txt"
with open(path,'r') as tweets:            ###opening original tweet file
    for line in tweets:                         
        line = line.strip()                       ##stripping characters before and after the line
        words = line.split()                      ## getting words from  the lines
        for word in words:
            word = word.lower()                 ##getting lower case of the word
            word = clean(word)                  ##applying clean function to the words
            print(word)
            if word not in stop_words:          ##check if word  is not in stop_words dictionary
                if word in positive_words:      ## check if the word is in positive_words dictionary 
                    print('+')
                    positive_count = positive_count +1       ###if yes add 1 to count of positive_tweets
                if word in negative_words:                    ##if no checck if the words is in negative_words dictionary
                    print('+')
                    negative_count = negative_count +1        ## if yes add 1 to the count of negative_tweets
                elif word in uncertain_words:
                    print('+')
                    uncertain_count = uncertain_count +1
                    
##concatenating string POSITIVE COUNT:with string of positive_sentiment
print("POSITIVE COUNT:" + str(positive_count))  
##concatenating string NEGATIVE COUNT:with string of negative_sentiment
print("NEGATIVE COUNT:" + str(negative_count)) 
##concatenating string UNCERTAIN COUNT:with string of uncertain_sentiment
print("UNCERTAIN COUNT:" + str(uncertain_count)) 

# Concatening and printing sum of positive and negative tweets:

print("Sentiment score:" + str((positive_count - negative_count) / (positive_count + negative_count)))

in
our
days
we
have
a
new
scene
of
superstitious
+
impostors
and
heretics,”
declared
the
scholastic
robert
burton,
about
the
fierce
+
english
calvinists
skeptical
+
of
the
anglican
compromise
and
known
by
a
series
of
sobriquets
in
the
unruly
+
englands
of
elizabeth
and
james.
they
referred
to
themselves
as
“the
godly,”
but
were
called
by
others
less
charitable
+
“the
precise;”
another
epithet
was
“brownist,”
after
robert
“troublechurch”
browne,
an
early
charismatic
+
leader.
they
were
known
also
as
separatists,
for
their
sporadic
+
+
opposition
+
to
the
anglican
church,
became
known
as
puritans,
for
their
theological
severity,
and
later
still
as
pilgrims,
in
recognition
of
the
atlantic
journey
that
their
theology
seemed
to
require
of
them,
and
that
brought
to
the
fertile
+
soil
of
mercantilist
america
the
ascetic
seedlings
of
our
vast
middleclassness.
“of
those
men
i
may
conclude
generally,”
burton
wrote
in
1621,
a
year
after
the
sailing
of
the
mayflower,
“that
howsoever
they
may
seem


nine
acres
of
english
farmland
for
a
full
year.
by
the
mid1630s,
a
single
beaver
hat
cost
five
pounds,
more
than
double
the
price
of
fifteen
years
before.
in
1628,
more
than
thirteen
hundred
pelts
from
north
american
beavers
arrived
in
england,
and
at
the
peak
of
the
trade
in
the
1630s,
the
pilgrims
were
delivering
more
than
two
thousand
skins
annually.
a
corner
had
been
turned:
the
pilgrims
were
rich.
but
how
had
that
happened
the
turnaround
in
colonial
fortunes
was
quick
and
decisive—especially
quick
in
bunker’s
excited
+
telling—but
it
was
far
from
inevitable.
what
distinguished
+
the
mayflower
pilgrims
from
their
virginia
cousins,
and
what
explains
their
special
willingness
+
to
penetrate
the
new
continent
in
search
of
game,
bunker
writes,
was
not
theology
so
much
as
genealogy.
to
be
sure,
the
puritans
possessed
what
bunker
calls
an
“evangelical
superego,”
but
they
also
possessed
an
unusual
+
+
and
unacknowledged
degree
of
comfort
+
with
precisely
+
the
kinds
of
problems,
and
preci

# Sentiment Analysis  with TextBlob and VADER Sentiment

In [59]:
##TextBlob is a Python (2 and 3) library for processing textual data. 
##It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.
##To install TextBlob (and associated linguistic data called corpora) -
# pip install textblob
# python -m textblob.download_corpora

##VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to 
##sentiments expressed in social media, and works well on texts from other domains.
##To install VADER Sentiment -
# pip install vaderSentiment

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## TextBlob

In [60]:
analysis1 = TextBlob(text)
analysis2 = TextBlob(fb_text1)
analysis3 = TextBlob(fb_text2)
analysis4 = TextBlob(nr_text)

With TextBlob, we get a polarity and a subjectivity metric.* The polarity is the sentiment itself, ranging from a -1 to a +1. The subjectivity is a measure of the sentiment being objective to subjective, and goes from 0 to 1. We'd rather see sentiment that is objective than subjective, so a lower score should likely denote a more likely-to-be-accurate reading. We'll see.

In [67]:
print(analysis1.sentiment)
print(analysis2.sentiment)
print(analysis3.sentiment)
print(analysis4.sentiment)

Sentiment(polarity=-0.13924242424242425, subjectivity=0.35439393939393943)
Sentiment(polarity=0.0070975056689342375, subjectivity=0.320453514739229)
Sentiment(polarity=0.15218855218855218, subjectivity=0.2282828282828283)
Sentiment(polarity=0.056015964796883574, subjectivity=0.43222441476858436)


## Vadersentiment

The neg is negative sentiment found, neu is anything found to be neutral, pos is positive, and the compound is "computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, then normalized... [it] is the most useful metric if you want a single unidimensional measure of sentiment." (from the docs). The docs also suggest:

positive sentiment: compound score >= 0.5  
neutral sentiment: -0.5 < compound score < 0.5  
negative sentiment: -0.5 <= compound score

In [66]:
analyzer = SentimentIntensityAnalyzer()
vs1 = analyzer.polarity_scores(text)
print(vs1)
vs2 = analyzer.polarity_scores(fb_text1)
print(vs2)
vs3 = analyzer.polarity_scores(fb_text2)
print(vs3)
vs4 = analyzer.polarity_scores(nr_text)
print(vs4)

{'neg': 0.059, 'neu': 0.892, 'pos': 0.05, 'compound': -0.1027}
{'neg': 0.024, 'neu': 0.814, 'pos': 0.162, 'compound': 0.91}
{'neg': 0.047, 'neu': 0.823, 'pos': 0.13, 'compound': 0.9118}
{'neg': 0.068, 'neu': 0.849, 'pos': 0.082, 'compound': 0.9883}
